In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from sklearn.pipeline import make_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [2]:
# labeling
# Get Dataset
files={
    'CICI':'/home/irteam/jiwlgus048-dcloud-dir/MLAC/data/encoded_ConcatedCICI.csv',
    'UNSW': '/home/irteam/jiwlgus048-dcloud-dir/MLAC/data/encoded_ConcatedUNSW.csv'
}

data = pd.read_csv(files['UNSW'])
data=data[np.isfinite(data).all(1)] # nan값 or 무한대값을 포함하지 않음

In [16]:
multiclass_labels=data['new_attack_category']
data=data.drop(labels=['unnamed_0_1','label','attack_category','new_attack_category'],axis=1)

In [19]:
# importance 높은 feature 삭제
del data['ct_srv_src']
del data['ct_srv_dst']
del data['ct_dst_ltm']
del data['ct_src_ltm']
del data['ct_src_dport_ltm']
del data['ct_dst_sport_ltm']
del data['ct_dst_src_ltm']

In [21]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [22]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','recall_mi','recall_ma','recall_we']+\
                 ['precision_mi','precision_ma','precision_we'])
eval_path='/home/irteam/jiwlgus048-dcloud-dir/MLAC/evaluation'

confusion_path='/home/irteam/jiwlgus048-dcloud-dir/MLAC/confusion_matrix/unsw_definition'
if os.path.isdir(confusion_path)==False:
    os.mkdir(confusion_path)
cnt=0

In [23]:
X_train,X_test,y_train,y_test=train_test_split(data,multiclass_labels,test_size=0.3, shuffle=True, stratify=multiclass_labels, random_state=34)

In [24]:
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue',pad_inches=0.5)
    plt.clf()

In [26]:
for name, model in models:

    model=make_pipeline(model)
    print('training start...'+name)
    start_time=time.time()
    model.fit(X_train,y_train)
    end_time=time.time()
    
    print("모델 training 소요 시간: {:.2f}초".format(end_time - start_time))


    #evaluation
    
    print('evaluation start...')
    start_time=time.time()
    y_pred=model.predict(X_test)
    end_time=time.time()
    print("모델 test 소요 시간: {:.2f}초".format(end_time - start_time))

    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, y_pred)
    f1_mi = f1_score(y_test, y_pred,average='micro')
    f1_ma = f1_score(y_test, y_pred,average='macro')
    f1_we = f1_score(y_test, y_pred,average='weighted')
    recall_mi = recall_score(y_test, y_pred, average='micro')
    recall_ma = recall_score(y_test, y_pred, average='macro')
    recall_we = recall_score(y_test, y_pred, average='weighted')
    precision_mi = precision_score(y_test, y_pred, average='micro')
    precision_ma = precision_score(y_test, y_pred, average='macro')
    precision_we = precision_score(y_test, y_pred, average='weighted')
    
    model_eval.append(acc)
    model_eval.append(f1_mi)
    model_eval.append(f1_ma)
    model_eval.append(f1_we)
    model_eval.append(recall_mi)
    model_eval.append(recall_ma)
    model_eval.append(recall_we)
    model_eval.append(precision_mi)
    model_eval.append(precision_ma)
    model_eval.append(precision_we)


    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,y_pred)
    plot_confusion_matrix(confusion,labels=['Benign', 'Brute Force', 'Dos', 'Fuzzers']+\
                          ['Generic', 'Heartbleed', 'Malware', 'Reconnaissance', 'Web Attack', 'unauthorized access'],title=name)


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'new_unsw.csv'),index=False)

training start...RF
모델 training 소요 시간: 18.37초
evaluation start...
모델 test 소요 시간: 2.74초
name:RF,acc:0.8440017357473379,f1_score:0.8440017357473379,0.09467514623390497,0.7735733548350366,recall:0.8440017357473379,0.10159446244879847,0.8440017357473379,precision:0.8440017357473379,0.18161332882273343,0.7705391020168926
training start...CART
모델 training 소요 시간: 92.65초
evaluation start...
모델 test 소요 시간: 1.98초
name:CART,acc:0.9754656710001771,f1_score:0.9754656710001771,0.528228318668113,0.9743179721295887,recall:0.9754656710001771,0.5352217114440825,0.9754656710001771,precision:0.9754656710001771,0.6934987189950486,0.9787895831776959
training start...NB
모델 training 소요 시간: 14.81초
evaluation start...
모델 test 소요 시간: 24.57초
name:NB,acc:0.8458537403057024,f1_score:0.8458537403057024,0.31638452888075885,0.77885453838123,recall:0.8458537403057024,0.3883839587622054,0.8458537403057024,precision:0.8458537403057024,0.36042273861901963,0.7293135984410981
training start...LDA
모델 training 소요 시간: 139.50초
